In [1]:
import ibmos2spark
# @hidden_cell
credentials = {
    'endpoint': 'https://s3.eu-geo.objectstorage.service.networklayer.com',
    'service_id': 'iam-ServiceId-7efbe802-cf47-4252-a5bb-dfc0e1b291fc',
    'iam_service_endpoint': 'https://iam.eu-de.bluemix.net/oidc/token',
    'api_key': 'j8ayzbE3wpe3Wt-0QLE2rFaSTqjnDolUo2vw7SWLNXP7'
}

configuration_name = 'os_ff46a710404449b7909df912ef988503_configs'
cos = ibmos2spark.CloudObjectStorage(sc, credentials, configuration_name, 'bluemix_cos')

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
# Since JSON data can be semi-structured and contain additional metadata, it is possible that you might face issues with the DataFrame layout.
# Please read the documentation of 'SparkSession.read()' to learn more about the possibilities to adjust the data loading.
# PySpark documentation: http://spark.apache.org/docs/2.0.2/api/python/pyspark.sql.html#pyspark.sql.DataFrameReader.json

df_data_1 = spark.read.json(cos.url('medium-sparkify-event-data.json', 'sparkify-donotdelete-pr-akwcx6xtarvcl3'))
df_data_1.take(5)

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20191018123317-0000
KERNEL_ID = 8fa3e70d-c376-4707-a5b6-5672aa777ad8


[Row(artist='Martin Orford', auth='Logged In', firstName='Joseph', gender='M', itemInSession=20, lastName='Morales', length=597.55057, level='free', location='Corpus Christi, TX', method='PUT', page='NextSong', registration=1532063507000, sessionId=292, song='Grand Designs', status=200, ts=1538352011000, userAgent='"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"', userId='293'),
 Row(artist="John Brown's Body", auth='Logged In', firstName='Sawyer', gender='M', itemInSession=74, lastName='Larson', length=380.21179, level='free', location='Houston-The Woodlands-Sugar Land, TX', method='PUT', page='NextSong', registration=1538069638000, sessionId=97, song='Bulls', status=200, ts=1538352025000, userAgent='"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"', userId='98'),
 Row(artist='Afroman', auth='Logged In', firstName='Maverick', gender='M', 

In [2]:
# import libraries
from pyspark.sql import SparkSession
from pyspark.ml.feature import RegexTokenizer, CountVectorizer, \
    IDF, StringIndexer, RegexTokenizer, VectorAssembler, Normalizer, StandardScaler
import pyspark.sql.functions as F
from pyspark.sql.types import IntegerType, StringType, FloatType
from pyspark.ml.feature import RegexTokenizer, VectorAssembler, Normalizer, StandardScaler
from pyspark.sql.functions import udf
from pyspark.sql.functions import desc, asc
from pyspark.sql.functions import sum as Fsum
from pyspark.sql.functions import concat, lit, avg, split, isnan, isnull, when, count, col, \
sum, mean, stddev, min, max, countDistinct,approx_count_distinct, size, collect_set, round
from pyspark.sql import Window


import seaborn as sns
import re
from datetime import datetime
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt

from pyspark.ml.feature import StringIndexer, VectorAssembler, Normalizer, StandardScaler, MinMaxScaler
from pyspark.ml.feature import OneHotEncoder, StringIndexer, OneHotEncoderEstimator
from pyspark.ml.classification import  RandomForestClassifier, LogisticRegression, LinearSVC, NaiveBayes, GBTClassifier
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.mllib.evaluation import BinaryClassificationMetrics, MulticlassMetrics

In [3]:
def create_dataframe_forML(df):
    def subfunc_cleaning(df):
        # Remove no id rows
        df = df.filter(df["userId"] != "")
        
        return df
        
    def subfunc_featureCreation(df):
        # Make a list of ids who churned
        churn_id_list = df[df.page == "Cancellation Confirmation"].select("userId").distinct().collect()
        churn_id_list = [x['userId'] for x in churn_id_list]
        create_churn_udf = udf(lambda userid: 1 if userid in churn_id_list else 0, IntegerType())
        df = df.withColumn("churn", create_churn_udf(df.userId))

        # Create new userAgent column
        def create_new_agent(userAgent):
            if userAgent == None:
                computer_os = None
            else:
                computer_os = userAgent.split()[1]
                computer_os = re.sub(r'[\W\s]','' ,computer_os)

            return  computer_os
        create_new_agent_udf = udf(create_new_agent, StringType())
        df = df.withColumn("os_system", create_new_agent_udf(df.userAgent))

        # Create new location column
        def create_new_location(location):
            if location == None:
                location_first = None
            else:
                location_first = location.split(',')[-1].split('-')[0].strip()

            return location_first

        create_new_location_udf = udf(create_new_location, StringType())
        df = df.withColumn("location_first", create_new_location_udf(df.location))

        # Create total number of sessionId column
        w = Window.partitionBy(df.userId)
        df = df.withColumn('total_sessionId', size(collect_set('sessionId').over(w)))

        # Create total number of itemInSession column
        df = df.withColumn('total_itemInSession', count('itemInSession').over(w))


      

        # Create last_access_time,first_access_time columns
        df = df.withColumn('last_access_time', max('ts').over(w))
        df = df.withColumn('first_access_time', min('ts').over(w))

        # Create last_level column
        df = df.withColumn('last_level',when(df.last_access_time == df.ts, df.level))


        # Create time difference column
        def calculate_time_after_id_creation(last_time, first_time):
            last_access_datetime = datetime.utcfromtimestamp(last_time / 1000)
            first_access_datetime = datetime.utcfromtimestamp(first_time / 1000)
            time_after_id_creation = last_access_datetime - first_access_datetime
            result = time_after_id_creation.total_seconds()/3600

            return result

        calculate_time_after_id_creation_udf = udf(calculate_time_after_id_creation, FloatType())
        df = df.withColumn("time_after_id_creation(hour)", calculate_time_after_id_creation_udf(df.last_access_time, df.first_access_time))
        df = df.withColumn("time_after_id_creation(hour)", round(col('time_after_id_creation(hour)')/1, 2))


        # convert timestamp to date (string)
        def create_numWeek(ts):
            return datetime.utcfromtimestamp(ts / 1000).strftime("%V")
        def create_numMonth(ts):
            return datetime.utcfromtimestamp(ts / 1000).strftime("%m")
        def create_numYear(ts):
            return datetime.utcfromtimestamp(ts / 1000).strftime("%Y")

        create_numWeek_udf = udf(create_numWeek, StringType())
        create_numMonth_udf = udf(create_numMonth, StringType())
        create_numYear_udf = udf(create_numYear, StringType())
        df = df.withColumn('num_week', create_numWeek_udf(col('ts')))
        df = df.withColumn('num_month', create_numMonth_udf(col('ts')))
        df = df.withColumn('num_year', create_numYear_udf(col('ts')))

        # Make a top_100 alltime artist list
        tmp_list = df.where(df.artist != "").groupby("artist").count().sort(col("count").desc()).collect()[0:100]
        top_100_alltime_artist_list = [row["artist"] for row in tmp_list]
        top_100_alltime_artist_list

        # Make udf to set 1 at churn column in every row of the chrun users 
        def create_top100_alltime(artist):
            if artist in top_100_alltime_artist_list:
                return 1
            else:
                return 0

        create_top100_alltime_udf = udf(create_top100_alltime, IntegerType())
        df = df.withColumn("top100_artist_alltime", create_top100_alltime_udf(df.artist))
        # Count total number of top 100
        w = Window.partitionBy(df.userId)
        df = df.withColumn('total_Top100_artist_alltime', sum('top100_artist_alltime').over(w))

        # Make a dictionary of a best seller song list of each week
        tmp_list = df.select("num_week").distinct().sort("num_week").collect()
        available_week_list = [row["num_week"] for row in tmp_list]
        available_week_list


    # Make a dictionary of a best seller song list of each week

        def create_dict_top100_song_week(available_week_list):
            dict_top100_song_week = dict()
            for week in available_week_list:
                top_100_week_song_list = df.where((df.artist != "") & (df.num_week == week)).groupby("song","num_week").count()\
                .sort(col("num_week"), col("count").desc()).collect()[0:100]
                top_100_week_song_list = [row['song'] for row in top_100_week_song_list]
                dict_top100_song_week[week] = top_100_week_song_list

            return dict_top100_song_week

        dict_top100_song_week = create_dict_top100_song_week(available_week_list)

        # Make a top_100_song_week list
        def create_top100_song_week(song, num_week):
            if song in dict_top100_song_week[num_week]:
                return 1
            else:
                return 0

        create_top100_song_week_udf = udf(create_top100_song_week, IntegerType())
        df = df.withColumn("top100_song_week", create_top100_song_week_udf(df.song, df.num_week))
        # Count total number of top 100_song_week
        w = Window.partitionBy(df.userId)
        df = df.withColumn('total_Top100_song_week', sum('top100_song_week').over(w))
        
        
        return df
    


    def subfunc_joinDf_featureExtraction(df):
        # Create other dataframes to be joined to df dataframe to get 
        # Number of thumb up/ thumb down/ advert/ add friend/ upgrade/ downgrade/ error/ logout
        thumbsup_df = df.where(df.page == 'Thumbs Up').groupBy("userId").agg(count("page").alias("total_thumbsup"))
        thumbsdown_df = df.where(df.page == 'Thumbs Down').groupBy("userId").agg(count("page").alias("total_thumbsdown"))
        advert_df =  df.where(df.page == 'Roll Advert').groupBy("userId").agg(count("page").alias("total_rolladvert"))
        addfriend_df =  df.where(df.page == 'Add Friend').groupBy("userId").agg(count("page").alias("total_addfriend"))
        addplaylist_df =  df.where(df.page == 'Add to Playlist').groupBy("userId").agg(count("page").alias("total_addplaylist"))
        sub_upgrade_df = df.where(df.page == 'Submit Upgrade').groupBy("userId").agg(count("page").alias("total_sub_upgrade"))
        sub_downgrade_df = df.where(df.page == 'Submit Downgrade').groupBy("userId").agg(count("page").alias("total_sub_downgrade"))
        error_df = df.where(df.page == 'Error').groupBy("userId").agg(count("page").alias("total_error"))
        logout_df = df.where(df.page == 'Logout').groupBy("userId").agg(count("page").alias("total_logout"))
        last_level_df = df[df.last_level != 'None'].select("userId","last_level")
        
        # Create total spent time
        spent_time_df = df.groupBy("userId", "sessionId").agg(max('ts').alias("max_ts_session"), 
                                                              min('ts').alias("min_ts_session")).orderBy("sessionId", ascending=True)
        def calculate_time_inSession(max_ts_session, min_ts_session):
            max_ts_session_datetime = datetime.utcfromtimestamp(max_ts_session / 1000)
            min_ts_session_datetime = datetime.utcfromtimestamp(min_ts_session / 1000)
            spent_time_session = max_ts_session_datetime - min_ts_session_datetime
            result = spent_time_session.total_seconds()/3600

            return result

        calculate_time_inSession_udf = udf(calculate_time_inSession, FloatType())
        spent_time_df = spent_time_df.withColumn("time_spent_Session_hour", 
                                                     calculate_time_inSession_udf(spent_time_df.max_ts_session, spent_time_df.min_ts_session))

        w = Window.partitionBy(spent_time_df.userId)
        spent_time_df = spent_time_df.withColumn("time_spent_Total_hour", sum(spent_time_df.time_spent_Session_hour).over(w))
        spent_time_df = spent_time_df.drop('max_ts_session','min_ts_session')
        spent_time_df = spent_time_df.withColumn('time_spent_Session_hour', round(col('time_spent_Session_hour')/1, 2))
        spent_time_df = spent_time_df.withColumn('time_spent_Total_hour', round(col('time_spent_Total_hour')/1, 2))
        spent_time_df_only_total = spent_time_df.drop('time_spent_Session_hour','sessionId').distinct()
        
        # Make df_new for machine learning
        df_new = df.select("userId",'gender', 'churn', 'os_system', 'location_first', 'total_sessionId', 'total_itemInSession',"time_after_id_creation(hour)", 
                           "total_Top100_artist_alltime", "total_Top100_song_week").dropna().drop_duplicates()
        df_new = df_new.join(thumbsup_df, 'userId', how='left').distinct()
        df_new = df_new.join(thumbsdown_df, 'userId', how='left').distinct()
        df_new = df_new.join(advert_df, 'userId', how='left').distinct()
        df_new = df_new.join(addfriend_df, 'userId', how='left').distinct()
        df_new = df_new.join(addplaylist_df, 'userId', how='left').distinct()
        df_new = df_new.join(sub_upgrade_df, 'userId', how='left').distinct()
        df_new = df_new.join(sub_downgrade_df, 'userId', how='left').distinct()
        df_new = df_new.join(error_df, 'userId', how='left').distinct()
        df_new = df_new.join(logout_df, 'userId', how='left').distinct()
        df_new = df_new.fillna(0, subset=['total_thumbsup','total_thumbsdown', 'total_rolladvert', 'total_addfriend', 'total_addplaylist', 'total_sub_upgrade', 
                                         'total_sub_downgrade', 'total_error', 'total_logout'])
        df_new = df_new.join(last_level_df, 'userId', how='left').distinct()
        df_new = df_new.join(spent_time_df_only_total, 'userId', how='left').distinct()
        
        df_new1 = df_new.withColumn("time_spent_Total_day", round(col('time_spent_Total_hour')/24, 2))
        df_new1 = df_new1.withColumn("time_after_id_creation(day)", round(col('time_after_id_creation(hour)')/24, 2))
        df_new1 = df_new1.withColumn("avg_total_sessionId_afterCreation", round(col('total_sessionId') / col('time_after_id_creation(day)'), 2))
        df_new1 = df_new1.withColumn("avg_itemInSession_afterCreation", round(col('total_itemInSession')/col('time_after_id_creation(day)'), 2))
        df_new1 = df_new1.withColumn("avg_thumbsup_afterCreation", round(col('total_thumbsup')/col('time_after_id_creation(day)'), 2))
        df_new1 = df_new1.withColumn("avg_thumbsdown_afterCreation", round((col('total_thumbsdown')/col('time_after_id_creation(day)')), 2))
        df_new1 = df_new1.withColumn("avg_rolladvert_afterCreation", round((col('total_rolladvert')/col('time_after_id_creation(day)')), 2))
        df_new1 = df_new1.withColumn("avg_addfriend_afterCreation", round((col('total_addfriend')/col('time_after_id_creation(day)')), 2))
        df_new1 = df_new1.withColumn("avg_addplaylist_afterCreation", round((col('total_addplaylist')/col('time_after_id_creation(day)')), 2))
        df_new1 = df_new1.withColumn("avg_error_afterCreation", round((col('total_error')/col('time_after_id_creation(day)')), 2))
        df_new1 = df_new1.withColumn("avg_logout_afterCreation", round((col('total_logout')/col('time_after_id_creation(day)')), 2))
        df_new1 = df_new1.withColumn("avg_total_Top100_artist_alltime", round((col('total_Top100_artist_alltime')/col('time_after_id_creation(day)')), 2))
        df_new1 = df_new1.withColumn("avg_total_Top100_song_week", round((col('total_Top100_song_week')/col('time_after_id_creation(day)')), 2))
        
        return df_new1
    
    df = subfunc_cleaning(df)
    print("Cleaning is done")
    df = subfunc_featureCreation(df)
    print("Feature creation is done")
    df_new1 = subfunc_joinDf_featureExtraction(df)
    print("Joining dataframes and feature extraction are done")
    
    return df_new1

df_ML = create_dataframe_forML(df_data_1)
df_ML.cache()
df_ML.take(1)

Cleaning is done
Feature creation is done
Joining dataframes and feature extraction are done


[Row(userId='100010', gender='F', churn=1, os_system='iPhone', location_first='CT', total_sessionId=2, total_itemInSession=137, time_after_id_creation(hour)=73.04, total_Top100_artist_alltime=18, total_Top100_song_week=12, total_thumbsup=4, total_thumbsdown=3, total_rolladvert=22, total_addfriend=3, total_addplaylist=1, total_sub_upgrade=0, total_sub_downgrade=0, total_error=0, total_logout=1, last_level='free', time_spent_Total_hour=7.01, time_spent_Total_day=0.29, time_after_id_creation(day)=3.04, avg_total_sessionId_afterCreation=0.66, avg_itemInSession_afterCreation=45.07, avg_thumbsup_afterCreation=1.32, avg_thumbsdown_afterCreation=0.99, avg_rolladvert_afterCreation=7.24, avg_addfriend_afterCreation=0.99, avg_addplaylist_afterCreation=0.33, avg_error_afterCreation=0.0, avg_logout_afterCreation=0.33, avg_total_Top100_artist_alltime=5.92, avg_total_Top100_song_week=3.95)]

In [4]:
df_ML.count()

448

In [5]:
df_ML.head(1)

[Row(userId='100010', gender='F', churn=1, os_system='iPhone', location_first='CT', total_sessionId=2, total_itemInSession=137, time_after_id_creation(hour)=73.04, total_Top100_artist_alltime=18, total_Top100_song_week=12, total_thumbsup=4, total_thumbsdown=3, total_rolladvert=22, total_addfriend=3, total_addplaylist=1, total_sub_upgrade=0, total_sub_downgrade=0, total_error=0, total_logout=1, last_level='free', time_spent_Total_hour=7.01, time_spent_Total_day=0.29, time_after_id_creation(day)=3.04, avg_total_sessionId_afterCreation=0.66, avg_itemInSession_afterCreation=45.07, avg_thumbsup_afterCreation=1.32, avg_thumbsdown_afterCreation=0.99, avg_rolladvert_afterCreation=7.24, avg_addfriend_afterCreation=0.99, avg_addplaylist_afterCreation=0.33, avg_error_afterCreation=0.0, avg_logout_afterCreation=0.33, avg_total_Top100_artist_alltime=5.92, avg_total_Top100_song_week=3.95)]

# Modeling
Split the full dataset into train, test, and validation sets. Test out several of the machine learning methods you learned. Evaluate the accuracy of the various models, tuning parameters as necessary. Determine your winning model based on test accuracy and report results on the validation set. Since the churned users are a fairly small subset, I suggest using F1 score as the metric to optimize.

### Make train, test and validation sets 

In [29]:
### Make train, test and validation sets 
train, test, validation = df_ML.randomSplit([0.7, 0.15, 0.15], seed = 44)
print(train.count())
print(test.count())
print(validation.count())

298
79
71


### Make Pipelines

In [30]:
# index and encode categorical features gender, level and state
stringIndexer_forGender = StringIndexer(inputCol="gender", outputCol="indexed_gender", handleInvalid = 'skip')
stringIndexer_forLast_level = StringIndexer(inputCol="last_level", outputCol="indexed_last_level", handleInvalid = 'skip')
stringIndexer_forLocation = StringIndexer(inputCol="location_first", outputCol="indexed_location", handleInvalid = 'skip')

encoder = OneHotEncoderEstimator(inputCols=["indexed_gender", "indexed_last_level", "indexed_location"],
                                       outputCols=["gender_feat", "last_level_feat", "location_feat"],
                                handleInvalid = 'keep')

# create vector for features
features = ['gender_feat', 'last_level_feat', 'location_feat', 'time_after_id_creation(day)','avg_total_sessionId_afterCreation','avg_itemInSession_afterCreation',
            'avg_thumbsup_afterCreation','avg_thumbsdown_afterCreation','avg_rolladvert_afterCreation','avg_addfriend_afterCreation',
            'avg_addplaylist_afterCreation','avg_error_afterCreation','avg_logout_afterCreation','avg_total_Top100_artist_alltime','avg_total_Top100_song_week']
assembler = VectorAssembler(inputCols=features, outputCol="assembled_features")
scaler = MinMaxScaler(inputCol="assembled_features" , outputCol="scaled_features")

# initialize random forest classifier
rf = RandomForestClassifier(featuresCol="scaled_features",labelCol="churn")
# initialize logistic regression
lr = LogisticRegression(maxIter=5,threshold=0.3)
# initialize N GBTClassifier
gbt = GBTClassifier(featuresCol="scaled_features",labelCol="churn", maxIter=5, maxDepth=3)

# assemble pipelines
pipeline_rf = Pipeline(stages = [stringIndexer_forGender, stringIndexer_forLast_level, stringIndexer_forLocation, encoder, assembler, scaler ,rf])
pipeline_lr = Pipeline(stages = [stringIndexer_forGender, stringIndexer_forLast_level, stringIndexer_forLocation, encoder, assembler, scaler ,lr])
pipeline_gbt =  Pipeline(stages = [stringIndexer_forGender, stringIndexer_forLast_level, stringIndexer_forLocation, encoder, assembler, scaler, gbt])

In [31]:
# random forest
##
starttime = datetime.now()
model_rf = pipeline_rf.fit(train)
pred_train_rf = model_rf.transform(train)
pred_test_rf = model_rf.transform(test)
pred_validation_rf = model_rf.transform(validation)

## train_set
predictionAndLabels_rf = pred_train_rf.rdd.map(lambda x: (float(x.prediction), float(x.churn)))
# Instantiate metrics object
metrics_rf = MulticlassMetrics(predictionAndLabels_rf )
# F1 score
print("F1 score of training set: ", metrics_rf.fMeasure())
print("Precision of training set: ", metrics_rf.precision(1))
print("Recall of training set: " , metrics_rf.recall(1))
print(metrics_rf.confusionMatrix().toArray())
print()

## test_set
predictionAndLabels_rf = pred_test_rf.rdd.map(lambda x: (float(x.prediction), float(x.churn)))
# Instantiate metrics object
metrics_rf = MulticlassMetrics(predictionAndLabels_rf)
# F1 score
print("F1 score of test set: ", metrics_rf.fMeasure())
print("Precision of test set: ", metrics_rf.precision(1))
print("Recall of test set: " , metrics_rf.recall(1))
print(metrics_rf.confusionMatrix().toArray())
print(datetime.now() - starttime)
print()

## validation_set
predictionAndLabels_rf = pred_validation_rf.rdd.map(lambda x: (float(x.prediction), float(x.churn)))
# Instantiate metrics object
metrics_rf = MulticlassMetrics(predictionAndLabels_rf)
# F1 score
print("F1 score of validation set: ", metrics_rf.fMeasure())
print("Precision of validation set: ", metrics_rf.precision(1))
print("Recall of validation set: ", metrics_rf.recall(1))
print(metrics_rf.confusionMatrix().toArray())
print(datetime.now() - starttime)

F1 score of training set:  0.912751677852349
Precision of training set:  0.9361702127659575
Recall of training set:  0.6567164179104478
[[228.   3.]
 [ 23.  44.]]

F1 score of test set:  0.8082191780821918
Precision of test set:  0.75
Recall of test set:  0.3333333333333333
[[53.  2.]
 [12.  6.]]
0:01:51.494839

F1 score of validation set:  0.835820895522388
Precision of validation set:  0.625
Recall of validation set:  0.38461538461538464
[[51.  3.]
 [ 8.  5.]]
0:02:22.723863


In [9]:
# # Logistic regression
# ##
# starttime = datetime.now()
# model_lr = pipeline_lr.fit(train)
# pred_train_lr = model_lr.transform(train)
# pred_test_lr = model_lr.transform(test)
# pred_validation_lr = model_lr.transform(validation)

# ## train_set
# predictionAndLabels_lr = pred_train_lr.rdd.map(lambda x: (float(x.prediction), float(x.churn)))
# # Instantiate metrics object
# metrics_lr = MulticlassMetrics(predictionAndLabels_lr)

# # F1 score
# print("F1 score of training set: ", metrics_lr.fMeasure())
# print("Precision of training set: ", metrics_lr.precision(1))
# print("Recall of training set: " , metrics_lr.recall(1))
# print(metrics_lr.confusionMatrix().toArray())
# print()

# ## test_set
# predictionAndLabels_lr = pred_test_lr.rdd.map(lambda x: (float(x.prediction), float(x.churn)))
# # Instantiate metrics object
# metrics_lr = MulticlassMetrics(predictionAndLabels_lr)
# # F1 score
# print("F1 score of test set: ", metrics_lr.fMeasure())
# print("Precision of test set: ", metrics_lr.precision(1))
# print("Recall of test set: ", metrics_lr.recall(1))
# print(metrics_lr.confusionMatrix().toArray())
# print(datetime.now() - starttime)
# print()

# ## validation_set
# predictionAndLabels_lr = pred_validation_lr.rdd.map(lambda x: (float(x.prediction), float(x.churn)))
# # Instantiate metrics object
# metrics_lr = MulticlassMetrics(predictionAndLabels_lr)
# # F1 score
# print("F1 score of validation set: " , metrics_lr.fMeasure())
# print("Precision of validation set:  " , metrics_lr.precision(1))
# print("Recall on validation set: ", metrics_lr.recall(1))
# print(metrics_lr.confusionMatrix().toArray())
# print(datetime.now() - starttime)

In [32]:
# GBT
##
starttime = datetime.now()
model_gbt = pipeline_gbt.fit(train)
pred_train_gbt = model_gbt.transform(train)
pred_test_gbt = model_gbt.transform(test)
pred_validation_gbt = model_gbt.transform(validation)

## train_set
predictionAndLabels_gbt = pred_train_gbt.rdd.map(lambda x: (float(x.prediction), float(x.churn)))
# Instantiate metrics object
metrics_gbt = MulticlassMetrics(predictionAndLabels_gbt)

# F1 score
print("F1 score of training set: ", metrics_gbt.fMeasure())
print("Precision of training set: ", metrics_gbt.precision(1))
print("Recall of training set: ", metrics_gbt.recall(1))
print(metrics_gbt.confusionMatrix().toArray())
print()

## test_set
predictionAndLabels_gbt = pred_test_gbt.rdd.map(lambda x: (float(x.prediction), float(x.churn)))
# Instantiate metrics object
metrics_gbt = MulticlassMetrics(predictionAndLabels_gbt)
# F1 score
print("F1 score of test set: ", metrics_gbt.fMeasure())
print("Precision of test set: " , metrics_gbt.precision(1))
print("Recall of test set: " , metrics_gbt.recall(1))
print(metrics_gbt.confusionMatrix().toArray())
print(datetime.now() - starttime)
print()

## valid_set
predictionAndLabels_gbt = pred_validation_gbt.rdd.map(lambda x: (float(x.prediction), float(x.churn)))
# Instantiate metrics object
metrics_gbt = MulticlassMetrics(predictionAndLabels_gbt)
# F1 score
print("F1 score of validation set: ", metrics_gbt.fMeasure())
print("Precision of validation set: ", metrics_gbt.precision(1))
print("Recall of validation set: ", metrics_gbt.recall(1))
print(metrics_gbt.confusionMatrix().toArray())
print(datetime.now() - starttime)

F1 score of training set:  0.9295302013422819
Precision of training set:  0.896551724137931
Recall of training set:  0.7761194029850746
[[225.   6.]
 [ 15.  52.]]

F1 score of test set:  0.863013698630137
Precision of test set:  0.8333333333333334
Recall of test set:  0.5555555555555556
[[53.  2.]
 [ 8. 10.]]
0:02:19.446047

F1 score of validation set:  0.8208955223880597
Precision of validation set:  0.5454545454545454
Recall of validation set:  0.46153846153846156
[[49.  5.]
 [ 7.  6.]]
0:02:49.543222
